![title](https://appirio.com/wp-content/uploads/2016/01/gmail-2014.png)

# Use Case - Gmail API

# Gmail API for sending emails

- Google provides several useful Python APIs for interacting with their services
- One of them is the Gmail API
- This API can be used to do everything that is available through the GUI
- Being able to send a mail from a Python script is often very useful, e.g. if you need to monitor something

# Installing the Google API

To install the Google API in python run the following command in the terminal:

    pip install --upgrade google-api-python-client

# Turning on the Gmail API

How to turn on the Gmail API is described [here](https://developers.google.com/gmail/api/quickstart/python)

# Importing necessary libs

In [1]:
from oauth2client import tools, client, file
from oauth2client.clientsecrets import InvalidClientSecretsError
import argparse
from httplib2 import Http
from apiclient import discovery
from email.mime.text import MIMEText
import base64
import os

# Defining some necessary params

- The **client secrets** file is the file you will download when you have finished activating the Gmail API. This file is somewhat similar to the ssh private key.
- The **credentials** file will be generated when you first connect to the Gmail service
- **scopes** defines which access level that you require to the google service. The different access levels for gmail can be found [here](https://developers.google.com/gmail/api/auth/scopes).

In [22]:
PATH_CLIENT_SECRETS = "C:\\Users\\peter.sandberg\\Development\\Python-Presentation\\examples\\resources\\client_secrets.json"
PATH_CREDENTIALS = "C:\\Users\\peter.sandberg\\Development\\Python-Presentation\\examples\\resources\\gmail-credentials.json"
SCOPES = "https://www.googleapis.com/auth/gmail.compose"

# Getting the credentials for the service

- The credentials is the object that is used to authorize you towards the service
- If it does not already exist, and the `client_secrets.json` file exist, the user will be promted to accept connecting to the service

In [7]:
def get_credentials():
    store = file.Storage(PATH_CREDENTIALS)
    credentials = store.get()
    if not credentials or credentials.invalid:
        try:
            flags = argparse.ArgumentParser(parents=[tools.argparser]).parse_args()
            flow = client.flow_from_clientsecrets(PATH_CLIENT_SECRETS, SCOPES)
            credentials = tools.run_flow(flow, store, flags)
        except InvalidClientSecretsError:
            return None
    return credentials

# Connecting to the service

- If the credentials can be created/accessed, the service object can be initialize
- The service object is the object that will be used for all operations towards the service

In [25]:
def init_service():
    credentials = get_credentials()
    if credentials:
        http = credentials.authorize(Http())
        return discovery.build('gmail', 'v1', http=http)
    return None

# Creating an email message object

- The email is created in the 'MIME' format
- Then, it will be encoded to something which may be passed thorugh an URL

In [23]:
def create_message(sender, to, subject, message_text):
    message = MIMEText(message_text)
    message['to'] = to
    message['from'] = sender
    message['subject'] = subject
    encoded_message = str(base64.urlsafe_b64encode(bytes(message.as_string(), "utf8")), 'utf-8')
    return {'raw': encoded_message}

# Sending the message

In [14]:
def send_message(service, message):
    if service:
        (service.users().messages().send(userId="me", body=message)).execute()

# Example - Let's send a mail!

In [26]:
sender = "peter.sandberg@visma.com"
to = input("Send to: ").strip()
subject = input("Subject: ").strip()
message_text = input("Message: ").strip()

message = create_message(sender, to, subject, message_text)
service = init_service()

send_message(service, message)

Send to: petthosan@gmail.com
Subject: Hello
Message: This is a test
